# Transfer Learning

In this notebook, you'll learn how to use pre-trained networks to solved challenging problems in computer vision. Specifically, you'll use networks trained on [ImageNet](http://www.image-net.org/) [available from torchvision](http://pytorch.org/docs/0.3.0/torchvision/models.html). 

ImageNet is a massive dataset with over 1 million labeled images in 1000 categories. It's used to train deep neural networks using an architecture called convolutional layers. I'm not going to get into the details of convolutional networks here, but if you want to learn more about them, please [watch this](https://www.youtube.com/watch?v=2-Ol7ZB0MmU).

Once trained, these models work astonishingly well as feature detectors for images they weren't trained on. Using a pre-trained network on images not in the training set is called transfer learning. Here we'll use transfer learning to train a network that can classify our cat and dog photos with near perfect accuracy.

With `torchvision.models` you can download these pre-trained networks and use them in your applications. We'll include `models` in our imports now.

In [1]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import matplotlib.pyplot as plt

import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
from torchvision import datasets, transforms, models

In [2]:
torch.cuda.is_available()

True

Most of the pretrained models require the input to be 224x224 images. Also, we'll need to match the normalization used when the models were trained. Each color channel was normalized separately, the means are `[0.485, 0.456, 0.406]` and the standard deviations are `[0.229, 0.224, 0.225]`.

In [42]:
data_dir = 'D:/beyond limits/AI/datasets/Cat_Dog_data/Cat_Dog_data'

# TODO: Define transforms for the training data and testing data
train_transforms = transforms.Compose([transforms.RandomResizedCrop(255),
                                       transforms.RandomHorizontalFlip(),
                                       transforms.Resize(224),
                                       transforms.ToTensor(),
                                       transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
                                      ])

test_transforms = transforms.Compose([transforms.Resize((224,224)),
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
                                     ])

# Pass transforms in here, then run the next cell to see how the transforms look
train_data = datasets.ImageFolder(data_dir + '/train', transform=train_transforms)
test_data = datasets.ImageFolder(data_dir + '/test', transform=test_transforms)

trainloader = torch.utils.data.DataLoader(train_data, batch_size=64, shuffle=True)
testloader = torch.utils.data.DataLoader(test_data, batch_size=64)

In [43]:
next(iter(train_data))[0].shape

torch.Size([3, 224, 224])

In [44]:
next(iter(test_data))[0].shape

torch.Size([3, 224, 224])

We can load in a model such as [DenseNet](http://pytorch.org/docs/0.3.0/torchvision/models.html#id5). Let's print out the model architecture so we can see what's going on.

In [45]:
model = models.densenet121(pretrained=True)
model

DenseNet(
  (features): Sequential(
    (conv0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (norm0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu0): ReLU(inplace=True)
    (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (denseblock1): _DenseBlock(
      (denselayer1): _DenseLayer(
        (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplace=True)
        (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (norm2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu2): ReLU(inplace=True)
        (conv2): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (denselayer2): _DenseLayer(
        (norm1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu

This model is built out of two main parts, the features and the classifier. The features part is a stack of convolutional layers and overall works as a feature detector that can be fed into a classifier. The classifier part is a single fully-connected layer `(classifier): Linear(in_features=1024, out_features=1000)`. This layer was trained on the ImageNet dataset, so it won't work for our specific problem. That means we need to replace the classifier, but the features will work perfectly on their own. In general, I think about pre-trained networks as amazingly good feature detectors that can be used as the input for simple feed-forward classifiers.

In [46]:
# Freeze parameters so we don't backprop through them
for param in model.parameters():
    param.requires_grad = False

from collections import OrderedDict
classifier = nn.Sequential(OrderedDict([
                          ('fc1', nn.Linear(1024, 500)),
                          ('relu', nn.ReLU()),
                          ('fc2', nn.Linear(500, 2)),
                          ('output', nn.LogSoftmax(dim=1))
                          ]))
    
model.classifier = classifier

With our model built, we need to train the classifier. However, now we're using a **really deep** neural network. If you try to train this on a CPU like normal, it will take a long, long time. Instead, we're going to use the GPU to do the calculations. The linear algebra computations are done in parallel on the GPU leading to 100x increased training speeds. It's also possible to train on multiple GPUs, further decreasing training time.

PyTorch, along with pretty much every other deep learning framework, uses [CUDA](https://developer.nvidia.com/cuda-zone) to efficiently compute the forward and backwards passes on the GPU. In PyTorch, you move your model parameters and other tensors to the GPU memory using `model.to('cuda')`. You can move them back from the GPU with `model.to('cpu')` which you'll commonly do when you need to operate on the network output outside of PyTorch. As a demonstration of the increased speed, I'll compare how long it takes to perform a forward and backward pass with and without a GPU.

In [21]:
import time

In [22]:
for device in ['cpu', 'cuda']:

    criterion = nn.NLLLoss()
    # Only train the classifier parameters, feature parameters are frozen
    optimizer = optim.Adam(model.classifier.parameters(), lr=0.001)

    model.to(device)

    for ii, (inputs, labels) in enumerate(trainloader):

        # Move input and label tensors to the GPU
        inputs, labels = inputs.to(device), labels.to(device)

        start = time.time()

        outputs = model.forward(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        if ii==3:
            break
        
    print(f"Device = {device}; Time per batch: {(time.time() - start)/3:.3f} seconds")

Device = cpu; Time per batch: 2.994 seconds
Device = cuda; Time per batch: 0.010 seconds


You can write device agnostic code which will automatically use CUDA if it's enabled like so:
```python
# at beginning of the script
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

...

# then whenever you get a new Tensor or Module
# this won't copy if they are already on the desired device
input = data.to(device)
model = MyModule(...).to(device)
```

From here, I'll let you finish training the model. The process is the same as before except now your model is much more powerful. You should get better than 95% accuracy easily.

>**Exercise:** Train a pretrained models to classify the cat and dog images. Continue with the DenseNet model, or try ResNet, it's also a good model to try out first. Make sure you are only training the classifier and the parameters for the features part are frozen.

In [47]:
## TODO: Use a pretrained model to classify the cat and dog images
model = models.densenet121(pretrained=True)
model

DenseNet(
  (features): Sequential(
    (conv0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (norm0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu0): ReLU(inplace=True)
    (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (denseblock1): _DenseBlock(
      (denselayer1): _DenseLayer(
        (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplace=True)
        (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (norm2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu2): ReLU(inplace=True)
        (conv2): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (denselayer2): _DenseLayer(
        (norm1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu

In [48]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [49]:
from collections import OrderedDict
my_classifier = nn.Sequential(OrderedDict([
                ('fc1', nn.Linear(1024, 500)),
                ('relu1', nn.ReLU()),
                ('fc2', nn.Linear(500, 64)),
                ('relu2', nn.ReLU()),
                ('fc3', nn.Linear(64, 2)),
                ('output', nn.LogSoftmax(dim=1))
]))

In [50]:
my_classifier

Sequential(
  (fc1): Linear(in_features=1024, out_features=500, bias=True)
  (relu1): ReLU()
  (fc2): Linear(in_features=500, out_features=64, bias=True)
  (relu2): ReLU()
  (fc3): Linear(in_features=64, out_features=2, bias=True)
  (output): LogSoftmax(dim=1)
)

In [51]:
# turning off gradient in pre-trained models
for param in model.parameters():
    param.requires_grad = False

In [52]:
model.classifier = my_classifier

In [57]:
criterion = nn.NLLLoss()
optimizer = optim.Adam(model.classifier.parameters(), lr=0.005)

In [58]:
# training models
model.to(device)
epochs = 5
training_losses, testing_losses, testing_accuracies = [], [], []

for epoch in range(epochs):
    running_loss = 0
    
    for images, labels in trainloader:
        optimizer.zero_grad()
        images, labels = images.to(device), labels.to(device)
        outputs = model.forward(images)
        loss = criterion(outputs, labels)
        running_loss += loss.item()
        loss.backward()
        optimizer.step()
    else:
        testing_loss = 0
        testing_accuracy = 0
        model.eval()
        with torch.no_grad():
            for test_images, test_labels in testloader:
                test_images, test_labels = test_images.to(device), test_labels.to(device)
                test_outputs = model.forward(test_images)
                test_loss = criterion(test_outputs, test_labels)
                testing_loss += test_loss.item()
                top_p, top_class = torch.topk(test_outputs, k=1)
                compare_output = top_class == test_labels.view(*top_class.shape)
                testing_accuracy += torch.mean(compare_output.type(torch.FloatTensor)).item()
        epoch_train_loss = running_loss / len(trainloader)
        epoch_test_loss = testing_loss / len(testloader)
        epoch_test_accuracy = testing_accuracy / len(testloader)
        print(f"Epoch: {epoch} Training loss: {epoch_train_loss}, Testing loss: {epoch_test_loss}, Testing Accuracy: {epoch_test_accuracy}")
        training_losses.append(epoch_train_loss)
        testing_losses.append(epoch_test_loss)
        testing_accuracies.append(epoch_test_accuracy)
        model.train()

Epoch: 0 Training loss: 0.0892951218993403, Testing loss: 0.03173176080445046, Testing Accuracy: 0.988671875
Epoch: 1 Training loss: 0.10361397403440523, Testing loss: 0.038354518274741164, Testing Accuracy: 0.98359375
Epoch: 2 Training loss: 0.09937892939937724, Testing loss: 0.04271661974705125, Testing Accuracy: 0.985546875
Epoch: 3 Training loss: 0.10269325074147094, Testing loss: 0.03439585898482278, Testing Accuracy: 0.985546875
Epoch: 4 Training loss: 0.10036008465050889, Testing loss: 0.03782894956020115, Testing Accuracy: 0.984765625


In [72]:
# using a resnet50 base
# using dense has a much better accuracy and lower loss

In [66]:
res_model = models.resnet50(pretrained=True)
res_model

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [67]:
# turning off gradient in pre-trained models
for param in res_model.parameters():
    param.requires_grad = False

In [68]:
from collections import OrderedDict
fc_classifier = nn.Sequential(OrderedDict([
                ('fc1', nn.Linear(2048, 500)),
                ('relu1', nn.ReLU()),
                ('fc2', nn.Linear(500, 64)),
                ('relu2', nn.ReLU()),
                ('fc3', nn.Linear(64, 2)),
                ('output', nn.LogSoftmax(dim=1))
]))

In [69]:
res_model.fc = fc_classifier

In [70]:
criterion = nn.NLLLoss()
optimizer = optim.Adam(model.classifier.parameters(), lr=0.005)

In [71]:
# training models
res_model.to(device)
epochs = 5
training_losses, testing_losses, testing_accuracies = [], [], []

for epoch in range(epochs):
    running_loss = 0
    
    for images, labels in trainloader:
        optimizer.zero_grad()
        images, labels = images.to(device), labels.to(device)
        outputs = res_model.forward(images)
        loss = criterion(outputs, labels)
        running_loss += loss.item()
        loss.backward()
        optimizer.step()
    else:
        testing_loss = 0
        testing_accuracy = 0
        res_model.eval()
        with torch.no_grad():
            for test_images, test_labels in testloader:
                test_images, test_labels = test_images.to(device), test_labels.to(device)
                test_outputs = res_model.forward(test_images)
                test_loss = criterion(test_outputs, test_labels)
                testing_loss += test_loss.item()
                top_p, top_class = torch.topk(test_outputs, k=1)
                compare_output = top_class == test_labels.view(*top_class.shape)
                testing_accuracy += torch.mean(compare_output.type(torch.FloatTensor)).item()
        epoch_train_loss = running_loss / len(trainloader)
        epoch_test_loss = testing_loss / len(testloader)
        epoch_test_accuracy = testing_accuracy / len(testloader)
        print(f"Epoch: {epoch} Training loss: {epoch_train_loss}, Testing loss: {epoch_test_loss}, Testing Accuracy: {epoch_test_accuracy}")
        training_losses.append(epoch_train_loss)
        testing_losses.append(epoch_test_loss)
        testing_accuracies.append(epoch_test_accuracy)
        res_model.train()

Epoch: 0 Training loss: 0.6905833889137615, Testing loss: 0.6897030830383301, Testing Accuracy: 0.516015625
Epoch: 1 Training loss: 0.6906433240933851, Testing loss: 0.6898003935813903, Testing Accuracy: 0.515625
Epoch: 2 Training loss: 0.6906839995221659, Testing loss: 0.6897764191031456, Testing Accuracy: 0.515625
Epoch: 3 Training loss: 0.6907270151105794, Testing loss: 0.6897205412387848, Testing Accuracy: 0.515234375
Epoch: 4 Training loss: 0.6905735797164115, Testing loss: 0.6894098907709122, Testing Accuracy: 0.516015625
